In [8]:
import sys
sys.path.append("../")

import pandas as pd
import os
import statsmodels.formula.api as sm
import numpy as np
import engarde.decorators as ed


from library import regulations
from library import characteristics
from library import analysis
from library import tables
from library import test_data

In [9]:
data_path = '/Users/kylieleblancKylie/domino/dofis/data/'
table_path = '/Users/kylieleblancKylie/domino/dofis/results/Who Needs Rules/'
data = pd.read_csv(os.path.join(data_path, 'clean', 'master_data_school.csv'),
                  sep=",", low_memory = False)
data = data[data.year == 2016]
data.head()

,Unnamed: 0,year,campus,campname,campischarter,district,distname,distischarter,rating_academic,rating_financial,...,elem,middle_math,middle_reading,middle_science,algebra,biology,eng1,math,reading,avescores
34304,34304,2016,1902001,CAYUGA H S,N,1902,CAYUGA ISD,N,M,Pass,...,NaN,NaN,NaN,NaN,0.724572,0.757077,0.855121,NaN,NaN,0.740825
34305,34305,2016,1902041,CAYUGA MIDDLE,N,1902,CAYUGA ISD,N,M,Pass,...,NaN,0.470379,0.722021,0.683798,NaN,NaN,NaN,0.470379,0.722021,0.608714
34306,34306,2016,1902103,CAYUGA EL,N,1902,CAYUGA ISD,N,M,Pass,...,0.982659,NaN,NaN,NaN,NaN,NaN,NaN,0.761413,1.203906,0.982659
34307,34307,2016,1903001,ELKHART H S,N,1903,ELKHART ISD,N,M,Pass,...,NaN,NaN,NaN,NaN,-0.451974,0.162768,0.647178,NaN,NaN,-0.144603
34308,34308,2016,1903041,ELKHART MIDDLE,N,1903,ELKHART ISD,N,M,Pass,...,NaN,-0.213925,0.277639,0.960365,0.939888,NaN,NaN,-0.213925,0.277639,0.261424


In [10]:
#TODO is -999 missing or NA? Worth differentiating?
data.doi_year.value_counts().sort_index()

2016.0      15
2017.0    2214
2018.0    3286
2019.0     727
2020.0      71
Name: doi_year, dtype: int64

# Geography

In [11]:
years = [2017, 2018, 2019]

In [12]:
labels = []
for char in characteristics.geography:
    labels.append(characteristics.labels[char])
    labels.append('')
table_dict = {'Characteristics': labels}
geo_table = pd.DataFrame(data=table_dict)

for yr in years:
    means = []
    df = data[data.doi_year == yr]
    for char in characteristics.geography:
        means.append(round(df[char].mean(), 2))
        sd = '[' + str(round(df[char].std(), 2)) + ']'
        means.append(sd)
    geo_table[yr] = means
    
geo_table

,Characteristics,2017,2018,2019
0,Urban,0.4,0.2,0.22
1,,[0.49],[0.4],[0.42]
2,Suburban,0.41,0.45,0.46
3,,[0.49],[0.5],[0.5]
4,Town,0.13,0.19,0.17
5,,[0.33],[0.4],[0.37]
6,Rural,0.06,0.15,0.15
7,,[0.23],[0.36],[0.36]


# Teacher characteristics

In [13]:
labels = []
for char in characteristics.teacher:
    labels.append(characteristics.labels[char])
    labels.append('')
table_dict = {'Characteristics': labels}
teacher_table = pd.DataFrame(data=table_dict)

for yr in years:
    means = []
    df = data[data.doi_year == yr]
    for char in characteristics.teacher:
        means.append(round(df[char].mean(), 2))
        sd = '[' + str(round(df[char].std(), 2)) + ']'
        means.append(sd)
    teacher_table[yr] = means
    
teacher_table

,Characteristics,2017,2018,2019
0,Ave. Experience Teaching,7.82,7.54,7.36
1,,[2.76],[2.68],[3.02]
2,Teacher Turnover Ratio,15.13,17.29,17.57
3,,[4.72],[6.2],[6.87]
4,Student-Teacher Ratio,14.57,14.03,14.09
5,,[3.92],[3.63],[6.13]


# Student 

In [14]:
labels = []
for char in characteristics.student:
    labels.append(characteristics.labels[char])
    labels.append('')
table_dict = {'Characteristics': labels}
student_table = pd.DataFrame(data=table_dict)

for yr in years:
    means = []
    df = data[data.doi_year == yr]
    for char in characteristics.student:
        means.append(round(df[char].mean(), 2))
        sd = '[' + str(round(df[char].std(), 2)) + ']'
        means.append(sd)
    student_table[yr] = means
    
student_table

,Characteristics,2017,2018,2019
0,Percent Hispanic,0.47,0.45,0.48
1,,[0.29],[0.28],[0.27]
2,Percent White,0.34,0.37,0.37
3,,[0.26],[0.29],[0.27]
4,Percent Black,0.12,0.13,0.09
5,,[0.14],[0.17],[0.11]
6,Percent Econ. Disadvantaged,0.57,0.61,0.56
7,,[0.29],[0.25],[0.25]
8,Average STAAR Performance (Std.),0.3,0.13,0.14
9,,[1.0],[0.83],[0.81]


# To Table

In [15]:
geo_table.columns

Index(['Characteristics', 2017, 2018, 2019], dtype='object')

In [16]:
dfs = [geo_table, teacher_table, student_table]
rows = [4, 13, 20]
for df, row in zip(dfs, rows):
    tables.df_to_excel(file = table_path + 'table2b_school_characteristics_by_year.xlsx', df = df,
                      df_columns = years, start_col = 2, start_row = row)

In [17]:
tables.n_to_excel(file = table_path + 'table2b_school_characteristics_by_year.xlsx',
                 col = 2, row = 31, n = len(data[data.doi_year == years[0]]))
tables.n_to_excel(file = table_path + 'table2b_school_characteristics_by_year.xlsx', 
                  col = 3, row = 31, n = len(data[data.doi_year == years[1]]))
tables.n_to_excel(file = table_path + 'table2b_school_characteristics_by_year.xlsx',
                 col = 4, row = 31, n = len(data[data.doi_year == years[2]]))